# Calculating ServiceArea using ArcGIS Location Services

A service area, also known as an isochrone, is a polygon that represents the distance that can be reached when driving or walking on a street network. This type of analysis is common in real estate search or determining the driving proximity to schools, businesses, or other facilities. For example, you can create a drive time polygon that represents how far you can drive in any direction from the center of a city in 20 minutes.

You can use service areas to build applications that:

- Visualize and measure the accessibility of locations that provide some kind of service. For example, a three-minute drive-time polygon around a grocery store can determine which residents are able to reach the store within three minutes and are thus more likely to shop there.

- By generating multiple service areas around one or more locations that can show how accessibility changes with an increase in travel time or travel distance. It can be used, for example, to determine how many hospitals are within 5, 10, and 15 minute drive times of schools.

- When creating service areas based on travel times, the service can make use of traffic data, which can influence the area that can be reached during different times of the day.

### What is ArcGIS Location Services?

The [ArcGIS Location Services](https://developers.arcgis.com/documentation/mapping-and-location-services/) are services hosted by Esri that provide geospatial functionality and data for building mapping applications. You can use the service APIs to display maps, access basemaps styles, visualize data, find places, geocode addresses, find optimized routes, enrich data, and perform other mapping operations. The services also support advanced routing operations such as fleet routing, calculating service areas, and solving location-allocation problems. To build applications you can use ArcGIS Maps SDKs, open source libraries, and scripting APIs.

### What You’ll Learn 

In this notebook you will be go over the steps for defining an UDF that invokes the Service Area endpoint, part of the ArcGIS Location Services. And perform the calculation for a set of warehouse addresses.

### Packages

This notebook requires the following packages to be added:
- pydeck

Let us start by configuring the variables as per your environment. These are:

- ESRI_API_KEY: The API key using which we can authenticate with the ArcGIS Location services api endpoints.
- DB_ROLE: The role that will be used to create and own the various objects For the purpose of the demo, I am going to keep it simple as to just use the ACCOUNTADMIN role.
- ARCGIS_DB: The database in which the tables, views, udf where the assets will be created.
- ARCGIS_DB_SCHEMA: A schema within the above database, to keep it simple, I am going to be using the default public schema.

Once you have configured these, run the cell. This cell will establish a snowflake session.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

#-----------------------
# Populate the below variables as per the environment
ESRI_API_KEY = 'AAPT85fOqywZsicJupSmVSCGrilHxUHWit8PKfw3XzfCwgtIjRzTYW7tM8tB7B0FgmpUwJz6Ql27EQeS-'
DB_ROLE = 'accountadmin'
ARCGIS_DB = 'arcgis_db'
ARCGIS_DB_SCHEMA = 'public'

#-----------------------
session.use_role(DB_ROLE)
session.use_database(ARCGIS_DB)
session.use_schema(ARCGIS_DB_SCHEMA)

### Defining the Servicearea UDF

The UDF will be reaching out to ARCGIS Location servicearea endpoint. It will also be needing the API key to access this. Hence we define the following objects:
 - secret: arcgis_api_key
 - network rule: nw_arcgis_api
 - external access integration: eai_arcgis_api
 - internal stage: lib_stg to store udf, as we are defining it as permanent

 Run the cell below, to create these objects

In [ ]:
sql_stmts = [
    f'use role {DB_ROLE}'
    ,f'use schema {ARCGIS_DB}.{ARCGIS_DB_SCHEMA}'
    
#   Create secret for holding ArcGis API Key
#   Ref: https://docs.snowflake.com/en/sql-reference/sql/create-secret
    ,f'''create or replace secret arcgis_api_key
        type = generic_string
        secret_string = '{ESRI_API_KEY}'
        comment = 'api key used for connecting to arcgis rest api endpoint.'
  '''

#   Create network rule
    ,f'''create or replace network rule {ARCGIS_DB}.{ARCGIS_DB_SCHEMA}.nw_arcgis_api
        mode = egress
        type = host_port
        value_list = ('*.arcgis.com')
        comment = 'Used for ESRI arcgis needs' '''

#   Create external access integration
    ,f''' create or replace external access integration eai_arcgis_api
        allowed_network_rules = (nw_arcgis_api)
        allowed_authentication_secrets = (arcgis_api_key)
        enabled = true
        comment = 'Used for ESRI arcgis needs' '''

#   Create internal stage
    ,f''' create stage if not exists {ARCGIS_DB}.{ARCGIS_DB_SCHEMA}.lib_stg
        encryption = (type = 'SNOWFLAKE_FULL' ) '''
  
]
for sql_stmt in sql_stmts:
  session.sql(sql_stmt).collect()

We define a Snowpark vectorized UDTF, that will be invoking the [ServiceArea API](https://developers.arcgis.com/rest/routing/serviceArea-service-direct/). 

As you would see the API can take a batch of geolocations and does require the input to be formatted in a specific format, we will be formatting the input accordingly.

While the service area has various optional parameter options, for this demo to keep it simple I am going to be using mainly the 'defaultBreaks' option. In this demo I am using 3 breaks (15, 30, 45). As a result the output from the API would also contain 3 service area, one for each breaks. Hence the implementation is UDTF rather than an UDF.

The response will contain the service area for each of the input location/facilities; hence we will be deconstructing the response into indivual service area/facility combination and returning them as the result.

In [ ]:
# define service area udf

import requests
import json
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
import pandas as pd
import copy

class ArcGIS_ServiceArea:
  def __init__(self):
    self.api_endpoint = 'https://route.arcgis.com/arcgis/rest/services/World/ServiceAreas/NAServer/ServiceArea_World/solveServiceArea'

  def _invoke_service_area_api(self, p_access_token ,p_facilities ,p_extra_params = {}):
    _headers = {
      'Authorization': f'Bearer {p_access_token}',
      'Content-Type': 'application/x-www-form-urlencoded'
    }
    _params = {
      'f' : 'json'
      # ,'token' : p_access_token
      ,'facilities' : json.dumps(p_facilities)
      ,**p_extra_params # dictionary unpacking
    }

    _response = requests.post(self.api_endpoint
                  ,data = _params 
                  ,headers = _headers)
    _response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    return _response

  def _build_facilities_payload(self, df):
    '''
    This function formats the input dataframe as per the API spec
    https://developers.arcgis.com/rest/routing/serviceArea-service-direct/#facilities
    '''
    # NOTE: Ensure Objectid is an integer, preferable sequence. otherwise the responses becomes invalid
    _features = []
    _facilityid_to_address_id_map = {}
    for idx ,row in df.iterrows():
      _facilityid_to_address_id_map[idx + 1] = row['address_id']
      _f = {
        "attributes": {
          "ObjectID" : idx + 1
          ,"Name" : row['address_id']
        },
        "geometry": {
          "x": row['x']
          ,"y": row['y']
        }
      }
      _features.extend([_f])
    _facilities = {
      'features' : _features
    }
    return (_facilityid_to_address_id_map ,_facilities)

  def _remap_response(self, p_facilityid_to_address_id_map ,p_response):
    '''
    This function remaps the response based on the objectid to address_id map
    '''
    _remapped_response = []
    for _f in p_response['saPolygons']['features']:
      _object_id = _f['attributes']['ObjectID']
      _facility_id = _f['attributes']['FacilityID']
      _address_id = p_facilityid_to_address_id_map.get(_facility_id, '-1')

      # Make a copy of the input response  
      _r_copy = copy.deepcopy( p_response )
      _r_copy['saPolygons']['features'] = [_f]

      _remapped_response.extend([
        {
          'address_id' : _address_id
          ,'object_id' : _object_id  
          ,'servicearea_response' : _r_copy
            
        }
      ])
    return _remapped_response

  def end_partition(self, df: T.PandasDataFrame[str,float ,float]) -> T.PandasDataFrame[str ,int ,dict]:
    import _snowflake # This is a private module that will be available during runtime.

    # Rename the columns
    df.columns = ['address_id' ,'x','y']

    # Extract the api from the secret
    _access_token = _snowflake.get_generic_secret_string('esri_api_key')

    _facilityid_to_address_id_map ,_facilities_payload = self._build_facilities_payload(df)
    # _travel_mode_payload = self._get_travel_mode()
    _additional_params = {
        'defaultBreaks' : '15,30,45'
        ,'preserveObjectID' : True
      }
    
    _response_payload = self._invoke_service_area_api(_access_token 
      ,_facilities_payload ,_additional_params)
    _response_payload.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

    # To store the formatted response based on defaultBreaks
    _vudtf_response = []

    # If the response is not 200, then we will just return the 
    # content asis for each input record, so that user can be aware of the error.
    # Another option is to log the event and raise an exception
    if _response_payload.status_code != 200:
    #if False: # For now, we will always process the response
      for idx ,row in df.iterrows():
        _vudtf_response.extend([
            _response_payload.json()
        ])
    else:
        _vudtf_response = self._remap_response(
          _facilityid_to_address_id_map, _response_payload.json())
      
    # Convert the list of geocoded values to a pandas dataframe
    r_df = pd.DataFrame(_vudtf_response)  
    return r_df

  end_partition._sf_vectorized_input = pd.DataFrame

# --------------------------------------------------------------------------------------------
# Ensure the current role and schema context
session.use_role(DB_ROLE)
session.use_database(ARCGIS_DB)
session.use_schema(ARCGIS_DB_SCHEMA)

# Register the snowpark UDTF
# Ref : https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.functions.pandas_udtf
fn_servicearea_for_addresses = F.pandas_udtf(
    ArcGIS_ServiceArea,
	output_schema = ['address_id' ,'object_id' ,'servicearea_response'],
		input_types = [
            T.PandasDataFrameType([T.StringType() ,T.FloatType() ,T.FloatType()])
        ], 
		input_names = ['"address_id"' ,'"x"' ,'"y"'],
    name = 'arcgis_servicearea_for_address_vudtf',
    replace=True, is_permanent=True, stage_location='@lib_stg',
    packages=['pandas', 'requests'],
    external_access_integrations=['eai_arcgis_api'],
    secrets = {
        'esri_api_key' : f'{ARCGIS_DB}.{ARCGIS_DB_SCHEMA}.arcgis_api_key'
    },
    max_batch_size = 100,
		comment = 'UDTF that takes a list of location geocode (latitude and longitutde) and returns the service area/isochrone from this point'
    )


The raw response and geometries returned from the API would not be usable by various mapping libraries, hence we need to extract geometries and also reformat to geojson format.
To do this, we define an UDF.

In [ ]:

def _convert_sapolygons_geometry_to_geojson(p_response: dict):
    # Random point
    _geojson =  {
        "coordinates": [
          -87.942989020543,
          46.259970794197244
        ],
        "type": "Point"
      }
    
    if 'saPolygons' not in p_response:
        return _geojson

    elif 'features' not in p_response['saPolygons']:
        return _geojson

    elif 'geometry' not in p_response['saPolygons']['features'][0]:
        return _geojson

    _g = p_response['saPolygons']['features'][0]['geometry']
    _rings = _g['rings']
    _geojson =  {
        "type": "MultiPolygon"
        ,"coordinates": [_rings]
    }

    return _geojson

def _extract_sapolygons_as_geojson(df :pd.DataFrame):

    _geojsons = []
    for idx ,row in df.iterrows():
        _sa_response = row[0] 
        _g = _convert_sapolygons_geometry_to_geojson(_sa_response)
        _geojsons.append(_g)

    r_df = pd.Series(_geojsons)
    return r_df

# Ref : https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.functions.pandas_udf
fn_extract_sapolygons_as_geojson = F.pandas_udf(
    func = _extract_sapolygons_as_geojson,
    return_type = T.PandasSeriesType(T.VariantType()),
    input_types=[T.PandasDataFrameType([T.VariantType()])],
    name = 'extract_sapolygons_as_geojson',
    replace=True, is_permanent=True,stage_location='@lib_stg',
    packages=['snowflake-snowpark-python'],
    max_batch_size = 100
)

---

## Demo data and sample execution

We now define some sample datasets and invoke the UDF's to invoke the service and extract the corresponding polygon geometries into its own specific columns.

When viewing the resulting table in ArcGISPro, a table with multiple geometry columns would not work. Hence we will be defining views on the table warehouses_serviceareas.

In [ ]:

-- 1. Create tables

create or replace transient table store_warehouses (
    address_id varchar
    ,address varchar
    ,latitude float
    ,longitude float
);

create or replace transient table warehouses_serviceareas (
    address_id varchar
    ,object_id integer
    ,address varchar
    ,address_pt geography
    ,servicearea_response variant
    ,sa_feature_attributes variant
    ,from_break int
    ,to_break int
    ,servicearea_isochrone geography
);


-- 1.1. Create a view for ArcGISPro
create or replace view vw_warehouses_serviceareas_serviceareas_feature as
select * exclude(address_pt ,servicearea_response)
from warehouses_serviceareas
;

create or replace view vw_warehouses_serviceareas_address_feature as
select distinct address_id, address ,address_pt
from warehouses_serviceareas
;

-- 1.2 Add search optimization for improve speed 
alter table warehouses_serviceareas
    add search optimization on geo(servicearea_isochrone);

alter table warehouses_serviceareas
    add search optimization on geo(address_pt);


-- 2. Ingest sample data
insert into store_warehouses values
('d56f6bc1328ab963f1462cb2d3830eb7','710 , Picaso Lane  ,Chico ,CA ,95926',	39.7474427,	-121.8656711)
,('d56f6bd199be0c5cea4f2461b3a391c4','Stellar Lp   ,Myrtle Beach ,SC ,29577',		33.6886227,	-78.9451313)
,('d56f6bd440a069311692b5a400098d0c','6816 , Southpoint Pkwy I   ,Jacksonville ,FL ,32216',		30.2575787,	-81.5890935)
,('d56f6bd9cb9d8d864647b4c86dab4b77','502 ,E Harris Street  ,Savannah ,GA ,31401',		32.07264,	-81.0882603)
,('d56f6c01502080a226ad897907e47bb0','1250 , Welch Road  ,Commerce Township ,MI ,48390',		42.545633,	-83.4578007)
,('d56f6c03b7dd6a972ea5b5b5b2cd8787','3 , Carlisle Street  ,Lancaster ,NY ,14086',		42.9291668,	-78.6594399)
,('d56f6c1bfa0618803d375c704650b5d4','25 ,E Delaware Parkway  ,Villas ,NJ ,08251',		39.0291768,	-74.932413)
,('d56f6c29924c88d893745ca5c97b28d5','65432 , 73rd Street  ,Bend ,OR ,97703',		44.1755873,	-121.2557281)
,('d56f6c3a921a017d32f5290f370a8e4e','1686 , Windriver Road  ,Clarksville ,TN ,37042',		36.6111711,	-87.3417787)
,('d56f6c628607c455e2753868907af75e','152 , Covey Rise Circle  ,Clarksville ,TN ,37043',		36.5659107,	-87.2297272)
;

In [ ]:
-- 3 invoke the UDTF to calculate the servicearea

select t.*
from store_warehouses as f
    ,table(arcgis_servicearea_for_address_vudtf(address_id ,longitude ,latitude) 
        over (partition by 1) ) as t

-- choose only those records to which the calculation was not done previously
where f.address_id not in (
    select distinct address_id from warehouses_serviceareas
)
;

-- 3.1 insert records into the serviceareas table 
merge into warehouses_serviceareas as t
using (
        select *
        from table(result_scan(last_query_id()))
    ) as s
on t.address_id = s.address_id
    and t.object_id = s.object_id
when not matched then insert
    (address_id ,object_id ,servicearea_response)
    values(s.address_id ,s.object_id ,s.servicearea_response)
;

-- sample output
select *
from warehouses_serviceareas
limit 1
;

----- 


In [ ]:
-- 4. Update feature attributes
update warehouses_serviceareas as l 
set
    sa_feature_attributes = l.servicearea_response:"saPolygons":features[0]:attributes
    ,address = r.address
    ,address_pt = st_makepoint(r.longitude ,r.latitude)
    ,from_break = l.servicearea_response:"saPolygons":features[0]:attributes:"FromBreak"::int 
    ,to_break = l.servicearea_response:"saPolygons":features[0]:attributes:"ToBreak"::int
from store_warehouses as r
where r.address_id = l.address_id
;


-- 4. convert the sa response and store it as geojson
update warehouses_serviceareas set
    servicearea_isochrone = try_to_geometry(
            extract_sapolygons_as_geojson(servicearea_response)
            ,4326 ,true
        )
;

In [ ]:
select *
from vw_warehouses_serviceareas_serviceareas_feature
limit 10;

---
## Visualization

While the dataset can be visualized using ArcGIS Pro; for a quick visualization we can do this using the PyDeck libraries as below. 

In [ ]:
import pydeck as pdk
import json

# The PyDeck does not have capability to handle GeoJson specifically with MultiPolygon
# Hence we need to extract the coordinates from its structure.
# Following that we have to flatten out the MultiPolygons as it cannot handle the nested
# nature.
#
# This operation can be done in python, but I find it easily doable at much speed when doing
# it in Snowflake. Hence the below statement does this extraction and flattening operations
#

sql_stmt = '''
with base as (
    select 
        address_id || '::' || object_id as addr_obj_id
        ,object_id
        ,address 
        ,from_break
        ,to_break
        ,st_x(address_pt) as lon
        ,st_y(address_pt) as lat
        ,st_asgeojson(servicearea_isochrone):type::varchar as geom_type
        ,st_asgeojson(servicearea_isochrone):coordinates as coordinates
        
    from warehouses_serviceareas
    
), polygon_coords as (
    select * 
    from base
    where geom_type = 'Polygon'
    
), multipolygon_coords as (
    select b.* exclude(coordinates) 
        ,f.value as coordinates
    from base as b
        ,lateral flatten(input => coordinates) as f
    where geom_type = 'MultiPolygon'

)
select * 
from polygon_coords
union all
select * 
from multipolygon_coords as b
-- where addr_obj_id like 'd56f6bc1328ab963f1462cb2d3830eb7%'
order by addr_obj_id
'''

spdf = session.sql(sql_stmt)
df = spdf.limit(50).to_pandas()

# Ensure COORDINATES is parsed to correct data type and not str
df['COORDINATES'] = df['COORDINATES'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

df.head()

In [ ]:
import random
import pydeck as pdk

# Filter the records to the selected address
tdf = df

# ----
#   Build initial view

# Take lat/lon from first address
_lon = tdf['LON'].iloc[0]
_lat = tdf['LAT'].iloc[0]
_initial_view_state = pdk.ViewState(
        latitude= _lat, longitude= _lon,
        zoom=10, max_zoom=16, 
        pitch=45, bearing=0
    )

# ----
#   Build layers
_deck_layers = []

# For each service area add a fill color Method 1: Using apply() with a lambda function (Recommended)
df['sa_fill_color'] = tdf.apply(lambda row: [random.randint(0, 255) ,random.randint(0, 255) ,random.randint(0, 255)], axis=1)

_l = pdk.Layer(
    "PolygonLayer",
    # data = df['COORDINATES_J'].to_list(), get_polygon='-',
    data = tdf, get_polygon='COORDINATES',
    get_fill_color = 'sa_fill_color',
    # get_fill_color = [random.randint(0, 255) ,random.randint(0, 255) ,random.randint(0, 255)],
    # get_line_color=[0, 0, 0, 255],
    pickable=True,
    auto_highlight=True,
    # filled=True,
    # extruded=True,
    # wireframe=True,
)
_deck_layers.append(_l)


# Add the address points
_address_pt_lyr = pdk.Layer(
            'ScatterplotLayer',
            data= tdf,
            get_position='[LON, LAT]',
            get_color=[0,0,0],
            get_radius=10,
            radiusScale=100,
            pickable=True)
_deck_layers.append(_address_pt_lyr)

# Build the pydeck map
tooltip = {
   "html": """ADDR : {ADDRESS} FromBreak : {FROMBREAK} OBJ ID: {OBJECT_ID}  """,
   "style": {
       "width":"10%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

_map_style= 'mapbox://styles/mapbox/streets-v11'

deck = pdk.Deck(
    layers = _deck_layers,
    map_style = _map_style,
    initial_view_state= _initial_view_state,
    tooltip = tooltip
)

# Visualize the polygons
st.pydeck_chart(deck)

---
## Finished!!!